# Test 2: Convergence Studies

Interactive spatial (manufactured solution) and temporal (self-convergence)
studies. Adjust surrogates, refinement levels, and time-step sequences.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from nitsche_signorini import SoilParams
from nitsche_signorini.runners import run_spatial_convergence, run_temporal_convergence

## Spatial convergence

In [ ]:
surrogates = {
    "D1": SoilParams(theta_s=0.4, theta_r=0.05, alpha=0.5, n=1.3, ell=0.5, K_s=1e-4),
    "D2": SoilParams(theta_s=0.4, theta_r=0.05, alpha=1.0, n=1.8, ell=0.5, K_s=1e-4),
    "C1": SoilParams(theta_s=0.4, theta_r=0.05, alpha=2.0, n=2.5, ell=0.5, K_s=1e-4),
    "C2": SoilParams(theta_s=0.4, theta_r=0.05, alpha=2.0, n=3.5, ell=0.5, K_s=1e-4),
}

n_levels = 5  # Adjustable
results = run_spatial_convergence(surrogates, n_levels=n_levels)

In [ ]:
# Print rates
norms = ['err_h', 'err_q', 'err_theta_L1', 'err_theta_L2']
for label in surrogates:
    r = results[label]
    print(f"\n{label}:")
    for k in norms:
        rates = [np.log2(r[k][i-1] / r[k][i]) if i > 0 else float('nan')
                 for i in range(n_levels)]
        print(f"  {k}: rates = {[f'{v:.2f}' for v in rates]}")

## Temporal convergence

In [ ]:
soil_d2 = SoilParams(theta_s=0.4, theta_r=0.05, alpha=1.0, n=1.8, ell=0.5, K_s=1e-4)

results_t = run_temporal_convergence(
    soil_d2, H=2.0, maxh=0.05,
    dt_levels=[240.0, 120.0, 60.0, 30.0],
    dt_ref=15.0,
    t_final=3600.0,
    p_rain=0.5 * soil_d2.K_s)

for i, r in enumerate(results_t):
    rh = np.log2(results_t[i-1]['err_h'] / r['err_h']) if i > 0 else float('nan')
    print(f"dt={r['dt']:.0f}s: err_h={r['err_h']:.4e}, rate={rh:.2f}")